In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torch.nn.functional as F
import os
import sys

In [26]:
inlist=[14,10,5]
intuple=[(inlist[i],inlist[i+1]) for i in range(len(inlist)-1)]
outlist=[ i for i in inlist[::-1]]
outtuple=[(outlist[i],outlist[i+1]) for i in range(len(outlist)-1)]
print(intuple,outtuple)
for (a,b) in intuple:
    print(a,b)

[(14, 10), (10, 5)] [(5, 10), (10, 14)]
14 10
10 5


In [41]:
# one hidden layer nn
class ANN(nn.Module):
    def __init__(self, input_dim, hidden_dim ,output_dim):
        super(ANN, self).__init__()
        self.fc1=nn.Linear(input_dim, hidden_dim)
        self.sigmoid=nn.Sigmoid()
        #uncomment if relu
        #self.sigmoid=nn.RelU()
        self.fc2=nn.Linear(hidden_dim, output_dim)
    def forward(self,x):
        out=self.fc1(x)
        #activate
        out=self.sigmoid(out)
        #uncomment if relu
        #out=self.ReLU(out)
        out=self.fc2(out)
        return out

In [48]:
ann=ANN(14,10,5)
#using view(-1 to bring in batch size)
ann(Variable(torch.rand(10,1,14).view(-1,14)))

Variable containing:
 0.1425  0.1747 -0.2461 -0.1618 -0.0922
 0.1124  0.1531 -0.2352 -0.1651 -0.0538
 0.1112  0.2070 -0.2484 -0.2053 -0.0685
 0.1244  0.1811 -0.2650 -0.1712 -0.0803
 0.1171  0.1983 -0.2251 -0.1936 -0.0820
 0.1208  0.1725 -0.2362 -0.1802 -0.0676
 0.1335  0.1446 -0.2219 -0.1264 -0.0538
 0.1223  0.2062 -0.2107 -0.2116 -0.0797
 0.1144  0.1841 -0.2595 -0.1779 -0.0535
 0.1223  0.1795 -0.2190 -0.1747 -0.0698
[torch.FloatTensor of size 10x5]

In [61]:
class MyModule(nn.Module):

    def __init__(self,intuple,outtuple):
        super(MyModule, self).__init__()
        self.ll=len(intuple)
        self.inLayers = nn.ModuleList([nn.Linear(a, b) for (a,b) in intuple])
        self.outLayers= nn.ModuleList([nn.Linear(a, b) for (a,b) in outtuple])
        for linear in self.inLayers:
            print(linear)
        for linear in self.outLayers:
            print(linear)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.inLayers):
            x=self.sigmoid(l(x))
            print(i, l)
        for i,l in enumerate(self.outLayers):
            if i<self.ll:
                x=self.sigmoid(l(x))
            else:
                x=l(x)
        return x

In [63]:
m=MyModule(intuple,outtuple)
m(Variable(torch.rand(10,1,14)).view(-1,14))

Linear (14 -> 10)
Linear (10 -> 5)
Linear (5 -> 10)
Linear (10 -> 14)
0 Linear (14 -> 10)
1 Linear (10 -> 5)


Variable containing:

Columns 0 to 9 
 0.5669  0.5473  0.5880  0.5426  0.5460  0.6287  0.4425  0.5735  0.5303  0.4521
 0.5669  0.5471  0.5881  0.5429  0.5461  0.6287  0.4425  0.5733  0.5303  0.4522
 0.5669  0.5472  0.5880  0.5427  0.5460  0.6287  0.4425  0.5735  0.5303  0.4522
 0.5669  0.5470  0.5881  0.5431  0.5462  0.6288  0.4424  0.5730  0.5303  0.4523
 0.5669  0.5473  0.5880  0.5428  0.5461  0.6288  0.4426  0.5734  0.5302  0.4522
 0.5669  0.5473  0.5880  0.5427  0.5460  0.6288  0.4426  0.5734  0.5303  0.4522
 0.5668  0.5471  0.5881  0.5430  0.5463  0.6288  0.4425  0.5730  0.5303  0.4523
 0.5669  0.5472  0.5880  0.5427  0.5460  0.6286  0.4425  0.5735  0.5302  0.4521
 0.5669  0.5473  0.5880  0.5426  0.5460  0.6287  0.4426  0.5736  0.5302  0.4521
 0.5669  0.5471  0.5881  0.5429  0.5462  0.6287  0.4424  0.5732  0.5303  0.4521

Columns 10 to 13 
 0.3954  0.3271  0.5146  0.5530
 0.3953  0.3272  0.5146  0.5529
 0.3953  0.3271  0.5146  0.5530
 0.3949  0.3272  0.5145  0.5529
 0.3953  0.3271